In [1]:
import ignite
import pandas as pd
import spacy
import torch

from collections import defaultdict
from itertools import chain

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from sklearn.model_selection import train_test_split
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext import data

from quoraquestionpairs.data import get_dataset
from quoraquestionpairs.neuralnets import RNNGRUSequential, RNNGRUParallel
from quoraquestionpairs.training import train

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
nlp = spacy.load('en')

In [4]:
question = data.Field(tokenize='spacy',
                      lower=True,
                      pad_first=True)

In [5]:
data_df = get_dataset('data/train.csv', question)

In [6]:
train_ds, val_ds = data_df.split(0.95, stratified=True, strata_field='is_duplicate')

In [7]:
question.build_vocab(train_ds,
                     min_freq=50)

In [9]:
train_iter = data.Iterator(train_ds,
                           batch_size=512,
                           repeat=False,
                           shuffle=True,
                           device=device)
    
val_iter = data.Iterator(val_ds,
                         batch_size=512,
                         device=device)

In [10]:
def prepare_batch(batch, device, non_blocking):
    (x1, x2), y = batch
    x1 = x1.to(device)
    x2 = x2.to(device)
    y = y.type(torch.cuda.FloatTensor).to(device)
    return (x1, x2), y

In [25]:
rnn = RNNGRUParallel(len(question.vocab), 300, 32, 1).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)

In [26]:
trainer = create_supervised_trainer(rnn, optimizer, criterion, prepare_batch=prepare_batch)
evaluator = create_supervised_evaluator(rnn, metrics={'accuracy': ignite.metrics.Accuracy(),
                                                      'loss': ignite.metrics.Loss(criterion)},
                                        prepare_batch=prepare_batch)

In [27]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_iter)
    metrics = evaluator.state.metrics
    print("Train Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_iter)
    metrics = evaluator.state.metrics
    print("Valid Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))

In [42]:
(x1, x2), y = next(iter(train_iter))

In [45]:
y.size()

torch.Size([94])

In [44]:
rnn((x1, x2)).size()

torch.Size([94])

In [28]:
trainer.run(train_iter, max_epochs=10)

Train Results - Epoch: 1  Avg accuracy: 0.78 Avg loss: 0.45
Valid Results - Epoch: 1  Avg accuracy: 0.77 Avg loss: 0.49
Train Results - Epoch: 2  Avg accuracy: 0.79 Avg loss: 0.44
Valid Results - Epoch: 2  Avg accuracy: 0.77 Avg loss: 0.47
Train Results - Epoch: 3  Avg accuracy: 0.79 Avg loss: 0.43
Valid Results - Epoch: 3  Avg accuracy: 0.77 Avg loss: 0.48
Train Results - Epoch: 4  Avg accuracy: 0.80 Avg loss: 0.43
Valid Results - Epoch: 4  Avg accuracy: 0.77 Avg loss: 0.47
Train Results - Epoch: 5  Avg accuracy: 0.80 Avg loss: 0.43
Valid Results - Epoch: 5  Avg accuracy: 0.77 Avg loss: 0.48
Train Results - Epoch: 6  Avg accuracy: 0.80 Avg loss: 0.43
Valid Results - Epoch: 6  Avg accuracy: 0.77 Avg loss: 0.48
Train Results - Epoch: 7  Avg accuracy: 0.80 Avg loss: 0.42
Valid Results - Epoch: 7  Avg accuracy: 0.77 Avg loss: 0.47
Train Results - Epoch: 8  Avg accuracy: 0.80 Avg loss: 0.42
Valid Results - Epoch: 8  Avg accuracy: 0.78 Avg loss: 0.47
Train Results - Epoch: 9  Avg accuracy: 